In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pickle
import sqlite3

In [ ]:
def load_cookie(driverx, path):
    with open(path, 'rb') as cookiesfile:
        cookies = pickle.load(cookiesfile)
        for cookie in cookies:
            driverx.add_cookie(cookie)

In [ ]:
conn = sqlite3.connect("mydatabase.db")
mycursor = conn.cursor()
create_table = "CREATE TABLE employees (Name_Surname text, Profile_Url text, Email text, Phone_Number text, City text, CV_Url text)"
mycursor.execute(create_table)
drop_table = "DROP TABLE employees"
#mycursor.execute(drop_table)

In [ ]:
def insert_into_bd(Name_surname, Profile_url, Email, Phone_number, City, Cv_url):
    sql = "INSERT INTO employees (Name_Surname, Profile_Url, Email, Phone_Number, City, CV_Url) VALUES (?, ?, ?, ?, ?, ?)"
    val = (Name_surname, Profile_url, Email, Phone_number, City, Cv_url)
    mycursor.execute(sql, val)
    conn.commit()

In [ ]:
driver = webdriver.Firefox(executable_path='C:\\Users\\yurak\\Desktop\\test\\geckodriver.exe')
driver.get("https://notebook.rabota.ua/employer/notepad/cvs?vacancyId=-1")
#pickle.dump( driver.get_cookies() , open("cookies.pkl","wb"))
load_cookie(driver, 'C:\\Users\\yurak\\Desktop\\test\\cookies.pkl')

In [ ]:
for page in range(1, 19):
    driver.get('https://notebook.rabota.ua/employer/notepad/cvs?vacancyId=-1&pg=' + str(page))
    soup = BeautifulSoup(driver.page_source, "html.parser")
    for profile in soup.find_all("a", class_="rua-p-t_16 rua-p-c-default ga_cv_view_cv"):
        profile_url = profile["href"]
        name_surname = profile.text
        driver.get(profile_url)
        soup_profile = BeautifulSoup(driver.page_source, "html.parser")
        #print(soup_profile.prettify())
        try:
            email = soup_profile.find("span", {"class":"rua-p-t_13", "id":"ctl00_centerZone_BriefResume1_ViewAttachedCV1_cvHeader_lblEmailValue"}).get_text()
        except AttributeError:
            email = soup_profile.find("span", {"class":"rua-p-t_13", "id":"ctl00_centerZone_BriefResume1_CvView1_cvHeader_lblEmailValue"}).get_text()
        try:
            phone_number = soup_profile.find("span", {"class":"rua-p-t_13", "id":"ctl00_centerZone_BriefResume1_CvView1_cvHeader_lblPhoneValue"}).get_text()
        except AttributeError:
            phone_number = ""
        try:
            city = soup_profile.find("span", {"class":"rua-p-t_13", "id":"ctl00_centerZone_BriefResume1_CvView1_cvHeader_lblRegionValue"}).get_text()
        except AttributeError:
            city = ""  
        cv_url = str(soup.find("a", class_="rua-p-t_16")["href"])
        #driver.get(cv_url)
        #email = soup_profile.select('#span[class="rua-p-t_13"]')
        #email = soup_profile.find("span", id_="ctl00_centerZone_BriefResume1_ViewAttachedCV1_cvHeader_lblEmailValue", class_="rua-p-t_13")
        #print(name_surname + "\n" + profile_url + "\n" + email +"\n"+ phone_number + "\n" + city + "\n" + cv_url)
        insert_into_bd(name_surname, profile_url, email, phone_number, city, cv_url)
#sql_select_all = "SELECT * FROM employees"
#mycursor.execute(sql_select_all)
#print(mycursor.fetchall())